In [ ]:
# from sklearn.tree.tree import BaseDecisionTree
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import numpy as np
from numpy import *
import time
from operator import itemgetter
from multiprocessing import Pool
import os
import pandas as pd

In [ ]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [ ]:
my_region = boto3.session.Session().region_name

bucket_name = 'cs205-final'
s3 = boto3.resource('s3')
# try:
#     if  my_region == 'us-east-1':
#         s3.create_bucket(Bucket=bucket_name)
#     else: 
#         s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
#     print('S3 bucket created successfully')
# except Exception as e:
#     print('S3 error: ',e)

In [ ]:
# Loads two datasets
healthy_uri = f"s3://{bucket_name}/healthy.tsv"
cancer_uri = f"s3://{bucket_name}/675_cancer.tsv"

healthy_df = pd.read_csv(healthy_uri, sep='\t')
cancer_df = pd.read_csv(cancer_uri, sep='\t')

# boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'healthy.tsv')).upload_file('healthy.tsv')
# healthy_data = sagemaker.inputs.TrainingInput(s3_data='s3://{}'.format(bucket_name), content_type='tsv')

# boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, '675_cancer.tsv')).upload_file('675_cancer.tsv')
# cancer_data = sagemaker.inputs.TrainingInput(s3_data='s3://{}'.format(bucket_name), content_type='tsv')

In [ ]:
s3_intermediate = f"s3://{bucket_name}/intermediate/"

In [ ]:
healthy_df.shape

(49196, 193)

In [ ]:
healthy_df.head()

,Gene ID,Gene Name,"HPSI0213i-koun_2, skin","HPSI0213i-nawk_55, skin","HPSI0313i-airc_2, skin","HPSI0313i-airc_66, skin","HPSI0313i-cesj_1, skin","HPSI0413i-corn_1, skin","HPSI0413i-coxy_3, skin","HPSI0413i-coxy_33, skin",...,"HPSI0913i-ruql_3, skin","HPSI0913i-vopm_2, skin","HPSI0913pf-ffdl, skin","HPSI1113i-fumb_2, skin","HPSI1113i-nibo_2, skin","HPSI1113i-nibo_3, skin","HPSI1113i-zuta_1, skin","HPSI1113i-zuta_3, skin","HPSI1114i-qibk_2, skin","HPSI1213i-foqj_2, skin"
0,ENSG00000000003,TSPAN6,44.0,113.0,68.0,153.0,104.0,83.0,96.0,117.0,...,117.0,138.0,31.0,94.0,189.0,131.0,118.0,80.0,83.0,111.0
1,ENSG00000000005,TNMD,0.9,2.0,4.0,6.0,5.0,0.7,7.0,5.0,...,5.0,13.0,NaN,10.0,21.0,3.0,8.0,5.0,5.0,15.0
2,ENSG00000000419,DPM1,117.0,191.0,126.0,259.0,262.0,225.0,272.0,241.0,...,276.0,194.0,195.0,205.0,279.0,226.0,227.0,168.0,287.0,254.0
3,ENSG00000000457,SCYL3,4.0,6.0,4.0,9.0,8.0,5.0,4.0,6.0,...,5.0,4.0,5.0,8.0,7.0,7.0,7.0,6.0,6.0,5.0
4,ENSG00000000460,C1orf112,8.0,23.0,10.0,31.0,18.0,14.0,11.0,20.0,...,24.0,12.0,3.0,12.0,18.0,16.0,22.0,19.0,19.0,18.0


In [ ]:
healthy_genes = healthy_df['Gene Name']

In [ ]:
healthy = healthy_df.drop(['Gene ID'], axis=1)

In [ ]:
healthy = healthy.set_index('Gene Name', drop=True)

In [ ]:
healthy_T = healthy.T

In [ ]:
healthy_T.head()

Gene Name,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AC012213.5,AC009301.1,LINC02689,AL157886.1,H2AQ1P,AL589743.8,AL355312.6,AC120036.5,AC018931.1,AL731559.1
"HPSI0213i-koun_2, skin",44.0,0.9,117.0,4.0,8.0,0.9,NaN,51.0,10.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.1,NaN,NaN,0.1
"HPSI0213i-nawk_55, skin",113.0,2.0,191.0,6.0,23.0,0.2,NaN,100.0,30.0,77.0,...,0.2,NaN,0.1,NaN,0.6,NaN,NaN,NaN,NaN,0.1
"HPSI0313i-airc_2, skin",68.0,4.0,126.0,4.0,10.0,0.8,NaN,78.0,16.0,46.0,...,NaN,0.2,NaN,NaN,NaN,0.2,0.1,NaN,NaN,0.1
"HPSI0313i-airc_66, skin",153.0,6.0,259.0,9.0,31.0,0.3,NaN,109.0,33.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"HPSI0313i-cesj_1, skin",104.0,5.0,262.0,8.0,18.0,0.2,NaN,117.0,39.0,62.0,...,0.1,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,0.1


In [ ]:
healthy_T = healthy_T.fillna(0)

In [ ]:
healthy_T.shape

(191, 49196)

In [ ]:
healthy_arr = healthy_T.to_numpy()

In [ ]:
cancer_df.shape

(57711, 624)

In [ ]:
VIM = GENIE3(healthy_arr[:,:5])

Tree method: RF
K: sqrt
Number of trees: 1000


running single threaded jobs
Gene 1/5...
Gene 2/5...
Gene 3/5...
Gene 4/5...
Gene 5/5...
Elapsed time: 7.40 seconds


In [ ]:
get_link_list(VIM, gene_names=list(healthy_genes.values[:5]),file_name='healthy_output_ranking.txt')

nInter = 0
nInter = 1
nInter = 2
nInter = 3
nInter = 4
nInter = 5
nInter = 6
nInter = 7
nInter = 8
nInter = 9
nInter = 10
nInter = 11
nInter = 12
nInter = 13
nInter = 14
nInter = 15
nInter = 16
nInter = 17
nInter = 18
nInter = 19


In [ ]:
# build and train the model
container = get_image_uri(boto3.Session().region_name,'xgboost', repo_version='1.0-1')  # loads the xgboost model

# check
# max_depth = np.sqrt(healthy_data.shape[0])

output_path = f"s3://{bucket_name}/output/"


hyperparameters = {'max_depth':7,
                   'eta':'0.2',
                   'gamma':'4',
                   'min_child_weight':'6',
                   'subsample':'0.7',
                   'objective':'binary:logistic',
                   'num_round':50
                  }
                  
# TODO We want shared-memory distribution on a single node for each gene
estimator = sagemaker.estimator.Estimator(image_uri = container,
                                         hyperparameters = hyperparameters,
                                         role = sagemaker.get_execution_role(),
                                         train_instance_count = 1,
                                         train_instance_type = 'm1.m5.2xlarge',
                                         train_volume_size = 5,
                                         output_path = output_path,
                                         train_use_spot_instance = True,
                                         train_max_run = 300,
                                         train_max_wait = 600)
# estimator.fit({'train':s3_input_train, 'validation':s3_input_test}) 

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
# deployment
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [ ]:
def compute_feature_importances(estimator):
#     if isinstance(estimator, BaseDecisionTree):
#         return estimator.tree_.compute_feature_importances(normalize=False)
#     else:
    importances = [e.tree_.compute_feature_importances(normalize=False)
                   for e in estimator.estimators_]
    importances = asarray(importances)
    # number of samples meeting these conditions / total number of samples
    return sum(importances,axis=0) / len(estimator)

def get_link_list(VIM,gene_names=None,regulators='all',maxcount='all',file_name=None):

    """Gets the ranked list of (directed) regulatory links.

    Parameters
    ----------

    VIM: numpy array
        Array as returned by the function GENIE3(), in which the element (i,j) is the score of the edge directed from the i-th gene to the j-th gene.

    gene_names: list of strings, optional
        List of length p, where p is the number of rows/columns in VIM, containing the names of the genes. The i-th item of gene_names must correspond to the i-th row/column of VIM. When the gene names are not provided, the i-th gene is named Gi.
        default: None

    regulators: list of strings, optional
        List containing the names of the candidate regulators. When a list of regulators is provided, the names of all the genes must be provided (in gene_names), and the returned list contains only edges directed from the candidate regulators. When regulators is set to 'all', any gene can be a candidate regulator.
        default: 'all'

    maxcount: 'all' or positive integer, optional
        Writes only the first maxcount regulatory links of the ranked list. When maxcount is set to 'all', all the regulatory links are written.
        default: 'all'

    file_name: string, optional
        Writes the ranked list of regulatory links to the file file_name.
        default: None



    Returns
    -------

    The list of regulatory links, ordered according to the edge score. Auto-regulations do not appear in the list. Regulatory links with a score equal to zero are randomly permuted. In the ranked list of edges, each line has format:

        regulator   target gene     score of edge
    """

    # Check input arguments
    if not isinstance(VIM,ndarray):
        raise ValueError('VIM must be a square array')
    elif VIM.shape[0] != VIM.shape[1]:
        raise ValueError('VIM must be a square array')

    ngenes = VIM.shape[0]

    if gene_names:
        if not isinstance(gene_names,(list,tuple)):
            raise ValueError('input argument gene_names must be a list of gene names')
        elif len(gene_names) != ngenes:
            raise ValueError('input argument gene_names must be a list of length p, where p is the number of columns/genes in the expression data')

    if regulators != 'all':
        if not isinstance(regulators,(list,tuple)):
            raise ValueError('input argument regulators must be a list of gene names')

        if gene_names is None:
            raise ValueError('the gene names must be specified (in input argument gene_names)')
        else:
            sIntersection = set(gene_names).intersection(set(regulators))
            if not sIntersection:
                raise ValueError('The genes must contain at least one candidate regulator')

    if maxcount != 'all' and not isinstance(maxcount,int):
        raise ValueError('input argument maxcount must be "all" or a positive integer')

    if file_name and not isinstance(file_name,str):
        raise ValueError('input argument file_name must be a string')



    # Get the indices of the candidate regulators
    if regulators == 'all':
        input_idx = range(ngenes)
    else:
        input_idx = [i for i, gene in enumerate(gene_names) if gene in regulators]

    # Get the non-ranked list of regulatory links
    vInter = [(i,j,score) for (i,j),score in ndenumerate(VIM) if i in input_idx and i!=j]

    # Rank the list according to the weights of the edges
    vInter_sort = sorted(vInter,key=itemgetter(2),reverse=True)
    nInter = len(vInter_sort)

    # Random permutation of edges with score equal to 0
    flag = 1
    i = 0
    while flag and i < nInter:
        print(f'nInter = {i}')
        (TF_idx,target_idx,score) = vInter_sort[i]
        if score == 0:
            flag = 0
        else:
            i += 1

    if not flag:
        items_perm = vInter_sort[i:]
        items_perm = random.permutation(items_perm)
        vInter_sort[i:] = items_perm

    # Write the ranked list of edges
    nToWrite = nInter
    if isinstance(maxcount,int) and maxcount >= 0 and maxcount < nInter:
        nToWrite = maxcount

    if file_name:

        outfile = open(file_name,'w')

        if gene_names:
            for i in range(nToWrite):
                (TF_idx,target_idx,score) = vInter_sort[i]
                TF_idx = int(TF_idx)
                target_idx = int(target_idx)
                outfile.write('%s\t%s\t%.6f\n' % (gene_names[TF_idx],gene_names[target_idx],score))
        else:
            for i in range(nToWrite):
                (TF_idx,target_idx,score) = vInter_sort[i]
                TF_idx = int(TF_idx)
                target_idx = int(target_idx)
                outfile.write('G%d\tG%d\t%.6f\n' % (TF_idx+1,target_idx+1,score))


        outfile.close()

    else:

        if gene_names:
            for i in range(nToWrite):
                (TF_idx,target_idx,score) = vInter_sort[i]
                TF_idx = int(TF_idx)
                target_idx = int(target_idx)
                print('%s\t%s\t%.6f' % (gene_names[TF_idx],gene_names[target_idx],score))
        else:
            for i in range(nToWrite):
                (TF_idx,target_idx,score) = vInter_sort[i]
                TF_idx = int(TF_idx)
                target_idx = int(target_idx)
                print('G%d\tG%d\t%.6f' % (TF_idx+1,target_idx+1,score))



def GENIE3(expr_data,gene_names=None,regulators='all',tree_method='RF',K='sqrt',ntrees=1000,nthreads=1):

    '''Computation of tree-based scores for all putative regulatory links.

    Parameters
    ----------

    expr_data: numpy array
        Array containing gene expression values. Each row corresponds to a condition and each column corresponds to a gene.

    gene_names: list of strings, optional
        List of length p, where p is the number of columns in expr_data, containing the names of the genes. The i-th item of gene_names must correspond to the i-th column of expr_data.
        default: None

    regulators: list of strings, optional
        List containing the names of the candidate regulators. When a list of regulators is provided, the names of all the genes must be provided (in gene_names). When regulators is set to 'all', any gene can be a candidate regulator.
        default: 'all'

    tree-method: 'RF' or 'ET', optional
        Specifies which tree-based procedure is used: either Random Forest ('RF') or Extra-Trees ('ET')
        default: 'RF'

    K: 'sqrt', 'all' or a positive integer, optional
        Specifies the number of selected attributes at each node of one tree: either the square root of the number of candidate regulators ('sqrt'), the total number of candidate regulators ('all'), or any positive integer.
        default: 'sqrt'

    ntrees: positive integer, optional
        Specifies the number of trees grown in an ensemble.
        default: 1000

    nthreads: positive integer, optional
        Number of threads used for parallel computing
        default: 1


    Returns
    -------
    An array in which the element (i,j) is the score of the edge directed from the i-th gene to the j-th gene. All diagonal elements are set to zero (auto-regulations are not considered). When a list of candidate regulators is provided, the scores of all the edges directed from a gene that is not a candidate regulator are set to zero.

    G1, G2, G3, G4, G5, ... GN
    num cols (genes) = N
    num rows (samples) = M
    k = sqrt(N)
    ntrees = 1000

    not even one gene, time complexity one tree = Nk logk
    one gene, time complexity of one RF = ntrees * Nk logk
    all genes, number of trees in all RF = ntrees * (N^2)k logk

    G1  G5  0.0342   1
    G1  G2  0.0324   1
    G2  G13 0.0274   0

    N^2 = 30,000^2 = 900,000,000

    Threshold: e.g. 0.3
    '''

    time_start = time.time()

    # Check input arguments
    if not isinstance(expr_data,ndarray):
        raise ValueError('expr_data must be an array in which each row corresponds to a condition/sample and each column corresponds to a gene')

    ngenes = expr_data.shape[1]

    if gene_names:
        if not isinstance(gene_names,(list,tuple)):
            raise ValueError('input argument gene_names must be a list of gene names')
        elif len(gene_names) != ngenes:
            raise ValueError('input argument gene_names must be a list of length p, where p is the number of columns/genes in the expr_data')

    if regulators != 'all':
        if not isinstance(regulators,(list,tuple)):
            raise ValueError('input argument regulators must be a list of gene names')

        if gene_names is None:
            raise ValueError('the gene names must be specified (in input argument gene_names)')
        else:
            sIntersection = set(gene_names).intersection(set(regulators))
            if not sIntersection:
                raise ValueError('the genes must contain at least one candidate regulator')

    if tree_method != 'RF' and tree_method != 'ET':
        raise ValueError('input argument tree_method must be "RF" (Random Forests) or "ET" (Extra-Trees)')

    if K != 'sqrt' and K != 'all' and not isinstance(K,int):
        raise ValueError('input argument K must be "sqrt", "all" or a stricly positive integer')

    if isinstance(K,int) and K <= 0:
        raise ValueError('input argument K must be "sqrt", "all" or a stricly positive integer')

    if not isinstance(ntrees,int):
        raise ValueError('input argument ntrees must be a stricly positive integer')
    elif ntrees <= 0:
        raise ValueError('input argument ntrees must be a stricly positive integer')

    if not isinstance(nthreads,int):
        raise ValueError('input argument nthreads must be a stricly positive integer')
    elif nthreads <= 0:
        raise ValueError('input argument nthreads must be a stricly positive integer')


    print('Tree method: ' + str(tree_method))
    print('K: ' + str(K))
    print('Number of trees: ' + str(ntrees))
    print('\n')


    # Get the indices of the candidate regulators
    if regulators == 'all':
        input_idx = list(range(ngenes))
    else:
        input_idx = [i for i, gene in enumerate(gene_names) if gene in regulators]


    # Learn an ensemble of trees for each target gene, and compute scores for candidate regulators
    VIM = zeros((ngenes,ngenes))

    if nthreads > 1:
        print('running jobs on %d threads' % nthreads)

        # list of list of paramaters, len is ngenes
        input_data = list()
        for i in range(ngenes):
            # Parameters of GENIE3 function, i refers to output_idx
            input_data.append( [expr_data,i,input_idx,tree_method,K,ntrees] )
            
        """
        5 instance, each instance 10,000 genes
        #####  #####  #####  #####  #####
        Instance 1: 1 - 10,000
        Instance 2: 10,001 - 20,000
        Instance 3: 20,001 - 30,000
        """

        # TODO change to SageMaker parallelism
        # https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-use-api.html
        # PARALLEL process targeting each gene
        pool = Pool(nthreads)
        alloutput = pool.map(wr_GENIE3_single, input_data)

        # len(alloutput) is ngenes
        for (i,vi) in alloutput:
            VIM[i,:] = vi

    else:
        print('running single threaded jobs')
        for i in range(ngenes):
            print('Gene %d/%d...' % (i+1,ngenes))

            vi = GENIE3_single(expr_data,i,input_idx,tree_method,K,ntrees)
            VIM[i,:] = vi


    VIM = transpose(VIM)

    time_end = time.time()
    print("Elapsed time: %.2f seconds" % (time_end - time_start))

    return VIM


# function for single thread
def wr_GENIE3_single(args):
    return([args[1], GENIE3_single(args[0], args[1], args[2], args[3], args[4], args[5])])


# In parallel, split up the output_idx
def GENIE3_single(expr_data,output_idx,input_idx,tree_method,K,ntrees):

    ngenes = expr_data.shape[1]

    # Expression of target gene, select column
    output = expr_data[:,output_idx]

    # Normalize output data
    output = output / std(output)

    # Remove target gene from candidate regulators
    input_idx = input_idx[:]
    if output_idx in input_idx:
        input_idx.remove(output_idx)
    expr_data_input = expr_data[:,input_idx]

    # Parameter K of the tree-based method
    if (K == 'all') or (isinstance(K,int) and K >= len(input_idx)):
        max_features = "auto"
    else:
        max_features = K

    if tree_method == 'RF':
        treeEstimator = RandomForestRegressor(n_estimators=ntrees,max_features=max_features)
    elif tree_method == 'ET':
        treeEstimator = ExtraTreesRegressor(n_estimators=ntrees,max_features=max_features)

    # Learn ensemble of trees
    treeEstimator.fit(expr_data_input,output)
#     expr_data_input_csv = np.savetxt(s3_intermediate + "expr_data_input.csv", expr_data_input, delimiter=",")
#     output_csv = np.savetxt(s3_intermediate + "output.csv", output, delimiter=",")
#     estimator.fit({'train':sagemaker.inputs.TrainingInput(expr_data_input_csv), 'validation':sagemaker.inputs.TrainingInput(output_csv)})

    # Compute importance scores
    feature_importances = compute_feature_importances(treeEstimator)
    vi = zeros(ngenes)

    # for each target, all the other genes
    vi[input_idx] = feature_importances

    return vi


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a3a1fdb0-4946-49ac-bb9f-c39b38835b36' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>